In [1]:
%load_ext autoreload
%autoreload 2
import pynamod
import numpy as np
import pandas as pd
import MDAnalysis as mda
import nglview as nv
import itertools
from numba import jit
import torch
import copy
from tqdm.auto import trange

from pynamod.energy.energy_constants import get_consts_olson_98,BDNA_step

from pynamod.external_forces.restraint import Restraint

/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [2]:
import warnings
warnings.filterwarnings('ignore')

### PDB Structure Analysis

In [44]:
nucl = pynamod.DNA_Structure_from_Atomic(pdb_id='7VDT',leading_strands=['I'],proteins=[])

Sending GET request to https://files.rcsb.org/download/7VDT.pdb to fetch 7VDT's pdb file as a string.


In [45]:
nucl.analyze_DNA()

In [46]:
nucl.analyze_protein(nucl.u.select_atoms('segid E K F B G C H D'))

In [ ]:
nucl.analyze_protein(nucl.u.select_atoms('segid A'),ref_index=5)

In [ ]:
nucl.step_params[1:20] = torch.from_numpy(BDNA_step[6:])

In [47]:
nucl.move_to_coord_center()
nucl.view_structure()

NGLWidget()

In [59]:
nucl.step_params

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e-01,  3.3530e+00, -0.0000e+00,  1.7020e+00,
          3.5666e+01],
        [ 0.0000e+00,  4.3600e

In [84]:
a = [1,2,3]

[2, 3]

In [ ]:
nucl2 = nucl[::-1]
nucl2.move_to_coord_center()

In [11]:
torch.ones(6)[::-1]

ValueError: step must be greater than zero

In [26]:
nucl.proteins[0].u.atoms.moment_of_inertia()

array([[ 1.65480920e+08,  1.74872783e+07,  6.38415941e+07],
       [ 1.74872783e+07,  2.28171315e+08, -1.29246582e+07],
       [ 6.38415941e+07, -1.29246582e+07,  1.45358671e+08]])

In [49]:
nucl.move_to_coord_center()
nucl.view_structure()

NGLWidget()

### Geom Params Testing

In [15]:
params.ref_frames.geom_class

In [40]:
params = nucl.geom_params
ori = params.origins.clone()
ref_frames = params.ref_frames.clone()
step_params = params.local_params.clone()
params.local_params[:10] = torch.tile(torch.from_numpy(BDNA_step[6:]),(10,1))
print((ori != params.origins).sum(), (ref_frames != params.ref_frames).sum())
params.local_params = step_params

mod_Tensor(402) mod_Tensor(1206)


In [48]:
params = nucl.geom_params
ori = params.origins.clone()
ref_frames = params.ref_frames.clone()
step_params = params.local_params.clone()
params.local_params[10] = torch.from_numpy(BDNA_step[6:])
print((ori != params.origins).sum(), (ref_frames != params.ref_frames).sum())
params.local_params = step_params

mod_Tensor(375) mod_Tensor(1125)


In [14]:
params.origins.shape

torch.Size([135, 1, 3])

In [73]:
torch.tile(torch.from_numpy(BDNA_step[6:]),(135,1))

tensor([[ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0000,  1.7020, 35.6660],
        [ 0.0000,  0.4360,  3.3530, -0.0

### Linear Structure Generation

In [6]:
dna_gen = pynamod.DNA_Structure_Generated('atcg'*15)

In [7]:
nucl_temp = pynamod.DNA_Structure_from_Atomic(pdb_id='3LZ0',leading_strands=['I'],proteins=[])
nucl_temp.analyze_DNA()
nucl_temp.analyze_protein()

Sending GET request to https://files.rcsb.org/download/3LZ0.pdb to fetch 3LZ0's pdb file as a string.


In [8]:
nucl.append_structures((dna_gen,nucl_temp))

In [9]:
nucl.view_structure()

NGLWidget()

In [383]:
nucl.steps_params[200] += 1

In [384]:
nucl.base_ref_frames[200]

array([[-2.92181192e-01,  7.10065252e-01, -6.40653955e-01,
         0.00000000e+00],
       [ 9.51284805e-01,  1.46840634e-01, -2.71099703e-01,
         0.00000000e+00],
       [-9.84244460e-02, -6.88654606e-01, -7.18378355e-01,
         0.00000000e+00],
       [-4.35482160e+01, -5.92701608e+01, -1.76076348e+02,
         0.00000000e+00]])

In [12]:
df = nucl.get_DataFrame()

In [94]:
a = torch.ones((3,3),dtype=torch.double)

In [98]:
b = torch.zeros((3,),dtype=torch.float)
a[0] = b

In [ ]:
nucl.

In [102]:
a.sign()

tensor([[0., 0., 0.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)

### Pytest

In [13]:
import pytest

In [14]:
!pytest pynamod --disable-warnings

============================= test session starts ==============================
platform linux -- Python 3.11.9, pytest-7.4.4, pluggy-1.0.0
rootdir: /home/_shared/_projects/2024_pynamod_update/package/PyNAMod
plugins: anyio-4.4.0
collecting ... ^C


# Ongoing

### Rotation Function

In [11]:
def rotate_origins(origins,rotation_matrix,ref_ori1,ref_ori2):
    moved_origins = origins - ref_ori1
    return np.matmul(moved_origins,rotation_matrix.T) + ref_ori2


def rotate_bp_frames(bp_frames,changed_step_frame,change_index):
    bp_frames = bp_frames.copy()
    changed_bp_frame = np.zeros((4,4))
    changed_bp_frame[3,:3],changed_bp_frame[:3,:3] = pynamod.bp_step_geometry.get_ori_and_mat_from_step_opt(*changed_step_frame,
                                                bp_frames[change_index-1,:3,:3],
                                                bp_frames[change_index-1,3,:3])
    
    rotation_matrix = np.matmul(changed_bp_frame[:3,:3],bp_frames[change_index,:3,:3].T)
    print(rotation_matrix,np.linalg.qr(rotation_matrix)[0],sep='\n')
    #print(np.matmul(changed_bp_frame[:3,:3],bp_frames[change_index,:3,:3].T),rotation_matrix)
    if change_index == len(bp_frames) - 1:
        bp_frames[change_index] = changed_bp_frame
        return bp_frames
    r_frames = np.matmul(rotation_matrix,bp_frames[change_index+1:,:3,:3])
    #bp_frames[change_index+1:,:3,:3] = abs(np.linalg.qr(r_frames)[0])*np.sign(r_frames)
    bp_frames[change_index+1:,:3,:3] = r_frames
    bp_frames[change_index+1:,3,:3] = rotate_origins(bp_frames[change_index+1:,3,:3],rotation_matrix,
                                                    bp_frames[change_index,3,:3],changed_bp_frame[3,:3])
    bp_frames[change_index] = changed_bp_frame
    return bp_frames

def test(dna_structure,steps=1000):
    frames = np.copy(dna_structure.base_ref_frames)
    
    for i in range(steps):
        change_step = np.random.randint(2,30)
        dna_structure.pairs_list[change_step].step_params += np.random.uniform(-0.5,0.5,6)
        frames = rotate_bp_frames(frames,dna_structure.pairs_list[change_step].step_params,change_step)
        if i%1000 == 0:
            print(np.mean(abs(frames[:,:3,:3] - dna_structure.base_ref_frames[:,:3,:3])),np.mean(np.linalg.norm(frames[:,3,:3]-dna_structure.base_ref_frames[:,3,:3],axis=1)))
    %timeit rotate_bp_frames(frames,dna_structure.pairs_list[change_step].step_params,change_step)

In [12]:
dna_gen = pynamod.DNA_structure_generated('atcg'*250)
test(dna_gen)

[[ 9.99997218e-01 -1.92701615e-04  2.35110257e-03]
 [ 2.03393866e-04  9.99989635e-01 -4.54836926e-03]
 [-2.35020172e-03  4.54883481e-03  9.99986892e-01]]
[[-9.99997218e-01  1.92701615e-04  2.35110257e-03]
 [-2.03393866e-04 -9.99989635e-01 -4.54836926e-03]
 [ 2.35020172e-03 -4.54883481e-03  9.99986892e-01]]
5.393349928391246e-16 1.721245278394144e-12
[[ 0.99995764 -0.00440197  0.00808311]
 [ 0.00443456  0.9999821  -0.00401781]
 [-0.00806528  0.00405349  0.99995926]]
[[-0.99995764  0.00440197  0.00808311]
 [-0.00443456 -0.9999821  -0.00401781]
 [ 0.00806528 -0.00405349  0.99995926]]
[[ 0.99993849  0.00632378 -0.00911185]
 [-0.00627037  0.99996306  0.00587815]
 [ 0.00914869 -0.00582065  0.99994121]]
[[-0.99993849 -0.00632378 -0.00911185]
 [ 0.00627037 -0.99996306  0.00587815]
 [-0.00914869  0.00582065  0.99994121]]
[[ 0.99999264  0.00132428  0.00360027]
 [-0.00135176  0.9999699   0.0076397 ]
 [-0.00359004 -0.00764451  0.99996434]]
[[-0.99999264 -0.00132428  0.00360027]
 [ 0.00135176 -0.99

In [2]:
nucl = pynamod.DNA_structure_from_atomic(pdb_id='3LZ0',leading_strands=['I'],proteins=[])
nucl.analyze_DNA()
nucl.move_to_coord_center()
test(nucl,2)

/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/pypdb/pypdb.py:497: DeprecationWarning: The `get_pdb_file` function within pypdb.py is deprecated.See `pypdb/clients/pdb/pdb_client.py` for a near-identical function to use
  warnings.warn(


Sending GET request to https://files.rcsb.org/download/3LZ0.pdb to fetch 3LZ0's pdb file as a string.


/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: MN
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/_shared/_projects/2024_pynamod_update/package/PyNAMod/pynamod/bp_step_geometry.py:143: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 1, 'C', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  bp_frames[i,3,:3],bp_frames[i,:3,:3]=get_ori_and_mat_from_step_opt(params[0],params[1],params[2],
/home/_shared/_projects/2024_pynamod_update/package/PyNAMod/pynamod/bp_step_geometry.py:143: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
  bp_frames[i,3,:3],bp_frames[i,:3,:3]=get_ori_and_mat_from_step_opt(params[0],params[1],params[2],


ZeroDivisionError: division by zero

In [13]:
def get_ori_and_mat_from_step_opt(shift,slide,rise,tilt,roll,twist,R1_exp,o1_exp):
    tilt=np.deg2rad(tilt)
    roll=np.deg2rad(roll)
    twist=np.deg2rad(twist)

    #x=np.array([1.,0.,0.])
    y=np.array([0.,1.,0.])
    z=np.array([0.,0.,1.])
    temp1=np.zeros((3,3))
    temp2=np.zeros((3,3))
    
    gamma=np.sqrt(tilt**2 + roll**2)

    #norm=length(np.array([tilt,roll,0]))
    roll_tilt_axis=np.array([tilt,roll,0])
    roll_tilt_axis/=length(roll_tilt_axis)
    phi=calc_phi(roll_tilt_axis)
    rmat_inmat(z,twist/2-phi,temp1)
    rmat_inmat(y,gamma/2,temp2)
    temp3=np.dot(temp1,temp2)
    rmat_inmat(z,phi,temp2)   
    
    rm=np.dot(temp3, temp2)
    
    rmat_inmat(y,gamma,temp2)
    temp3=np.dot(temp1,temp2)
    rmat_inmat(z,twist/2+phi,temp2)  
    
    r2=np.dot(temp3, temp2)
    
    o2=np.dot(np.array([shift,slide,rise]),rm.T)
    R2_exp = np.dot(R1_exp,r2)
    o2_exp=o1_exp+np.dot(o2,R1_exp.T)
    return(o2_exp,R2_exp)

def rebuild_by_full_par_frame_numba(full_par_frame,start_bp_frame=None):
    
    length=full_par_frame.shape[0]
    bp_frames = np.zeros((length,4,4))
    
    if start_bp_frame is None:
        R1_exp = np.identity(3)
        o1_exp = np.array([0.0,0.0,0.0])
        bp_frames[0,:3,:3] = R1_exp
        bp_frames[0,3,:3] = o1_exp
    else:
        bp_frames[0] = start_bp_frame
        
    for i in range(1,length):
        params=full_par_frame[i]
        bp_frames[i,3,:3],bp_frames[i,:3,:3]=get_ori_and_mat_from_step_opt(params[0],params[1],params[2],
                                                                           params[3],params[4],params[5],
                                                                           bp_frames[i-1,:3,:3],bp_frames[i-1,3,:3])

    return(bp_frames)

In [14]:
def get_trig(angle):
    return angle.cos(),angle.sin()

def get_r_mat(angle1,angle2,angle3):
    cos1,sin1 = get_trig(angle1)
    cos2,sin2 = get_trig(angle2)
    cos3,sin3 = get_trig(angle3)
    
    a = sin1*sin3
    b = cos1*cos3
    c = sin1*cos3
    d = cos1*sin3
    chunk1 = torch.stack([b,-d,
                       c,-a
                      ],dim=1)
    chunk1 = chunk1*cos2.reshape(-1,1) + torch.stack([-a,-c,d,b],dim=1)
    
    chunk2 = torch.stack([cos1,sin1,-cos3,sin3],dim=1) * sin2.reshape(-1,1)
    r_mat = torch.cat([chunk1,chunk2,cos2.reshape(-1,1)],dim=1)[:,[0,1,4,2,3,5,6,7,8]]
    
    return r_mat.reshape(-1,3,3)

def rebuild_base_ref_frames(steps_frames,bp_frames):
    bp_frames = bp_frames.double()
    steps_frames = torch.clone(steps_frames)

    steps_frames[1:,3:] = torch.deg2rad(steps_frames[1:,3:])


    gamma = torch.norm(steps_frames[1:,3:5],dim=1)
    cos_phi =  steps_frames[1:,4]/gamma
    phi = torch.arccos(cos_phi)
    phi[steps_frames[1:,3]<0] *= -1
    rm = get_r_mat(steps_frames[1:,5]/2 - phi,gamma/2,phi).double()

    r2 = get_r_mat(steps_frames[1:,5]/2 - phi,gamma,steps_frames[1:,5]/2 + phi).double()
    o2 = torch.matmul(steps_frames[1:,:3].reshape(-1,1,3),torch.transpose(rm,2,1))
    bp_frames[0,:3,:3] = torch.eye(3).double()
    for i,r in enumerate(r2):
        bp_frames[i+1,:3,:3] = torch.mm(bp_frames[i,:3,:3],r)


    bp_frames[1:,3,:3] = torch.cumsum(torch.matmul(o2,torch.transpose(bp_frames[:-1,:3,:3],2,1)),axis=0).reshape(-1,3)
    
    return bp_frames

In [93]:
a = torch.ones(6)
b = torch.deg2rad(a[3:])
b *= 4
a

tensor([1., 1., 1., 1., 1., 1.])

In [92]:
b

tensor([0.0698, 0.0698, 0.0698])

In [132]:
def get_single_r_mat(angle1,angle2,angle3,R):
    cos1,sin1 = get_trig(angle1)
    cos2,sin2 = get_trig(angle2)
    cos3,sin3 = get_trig(angle3)
    
    a = sin1*sin3
    b = cos1*cos3
    c = sin1*cos3
    d = cos1*sin3
    
    R[0,0] = b*cos2 - a
    R[0,1] = -d*cos2 - c
    R[1,0] = c*cos2 + d
    R[1,1] = -a*cos2 + b
    R[0,2] = cos1*sin2
    R[1,2] = sin1*sin2
    R[2,0] = -cos3*sin2
    R[2,1] = sin3*sin2
    R[2,2] = cos2

def get_R_and_ori(steps_frame,R1,ori1):
    steps_frame = torch.clone(steps_frame)
    torch.deg2rad(steps_frame[3:],out=steps_frame[3:])
    gamma = torch.linalg.norm(steps_frame[3:5])
    cos_phi = steps_frame[4]/gamma
    phi = cos_phi.arccos()
    if steps_frame[3]<0:
        phi *= -1
    r2 = torch.clone(R1)
    get_single_r_mat(steps_frame[5]/2 - phi,gamma,steps_frame[5]/2 + phi,r2)
    
    rm = torch.clone(R1)
    get_single_r_mat(steps_frame[5]/2 - phi,gamma/2,phi,rm)

    o2 = torch.mm(steps_frame[:3].reshape(1,3),rm.T)
    
    changed_R = torch.mm(R1,r2)
    changed_ori = torch.mm(o2.reshape(1,3),R1.T) + ori1
    
    return changed_R,changed_ori

In [172]:
def rotate_R(R_frames,rot_matrix):
    R_frames = rot_matrix.reshape(1,3,3).matmul(R_frames)
    if torch.linalg.norm(R_frames,axis=1).mean() != 1:
        R_frames = abs(torch.linalg.qr(R_frames).Q)*R_frames.sign()
    
    return R_frames

def transform_ori(ori,rot_matrix,old_ori,changed_ori):
    ori -= old_ori
    ori = ori.matmul(rot_matrix.T) + changed_ori
    
    return ori

def rotate_bp_frames(bp_frames,changed_step_frame,change_index):
    bp_frames = torch.clone(bp_frames)

    changed_R,changed_ori = get_R_and_ori(changed_step_frame,bp_frames[change_index-1,:3,:3],bp_frames[change_index-1,3,:3])
    
    rot_matrix = changed_R.mm(bp_frames[change_index,:3,:3].T)
    
    bp_frames[change_index:,:3,:3] = rotate_R(bp_frames[change_index:,:3,:3],rot_matrix)
    bp_frames[change_index+1:,3,:3] = transform_ori(bp_frames[change_index+1:,3,:3],rot_matrix,
                                                    bp_frames[change_index,3,:3],changed_ori)
    bp_frames[change_index,3,:3] = changed_ori

    
    return bp_frames

In [168]:
%timeit rotate_R(R_frames,rot_matrix)

67.8 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [99]:
%timeit transform_ori(ori,rot_matrix,old_ori,changed_ori)

16.9 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [109]:
%timeit get_R_and_ori(step,rot_matrix,ori)

397 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [133]:
%timeit get_single_r_mat(a1,a2,a3,r)

108 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [74]:
cuda_frames = bp_frames.double().to('cuda')
cuda_params = params[3].double().to('cuda')

In [157]:
%timeit rotate_bp_frames(frames,step,3)

676 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [50]:
def test(dna_structure,test_steps=100001,compare_every=10000):
    frames = torch.clone(torch.from_numpy(dna_structure.base_ref_frames)).double()
    steps = torch.clone(torch.from_numpy(dna_structure.steps_params)).double()
    for i in trange(test_steps):
        change_step = torch.randint(2,30,(1,)).item()
        change = torch.rand((6,))-0.5
        steps[change_step] += change
        frames = rotate_bp_frames(frames,steps[change_step],change_step)
        if i%compare_every == 0:
            exp_frames = rebuild_base_ref_frames(steps,torch.zeros(frames.shape[0],4,4).double())
            dif = (exp_frames-frames)[-100:]
            print(torch.mean(abs(dif)[:,:3,:3]).item(),
                  torch.mean(torch.linalg.norm(dif[:,3,:3],dim=1)).item())
            

In [171]:
test(nucl,test_steps=101,compare_every=10)

  0%|          | 0/101 [00:00<?, ?it/s]

tensor(1.0000, dtype=torch.float64)
1.4683540359685912e-15 7.145045985308867e-13
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
2.1671483569876384e-15 1.45961092152047e-12
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
4.428076226487307e-15 2.1878521485216455e-12
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
1.0402599643956807e-14 4.315429962400446e-12
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
1.3369921730303923e-14 2.6604232633778127e-12
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtyp

In [59]:
long_dna = pynamod.DNA_structure_generated('atcg'*25000)

In [62]:
test(long_dna,test_steps=1001,compare_every=1000)

  0%|          | 0/1001 [00:00<?, ?it/s]

9.93919446899622e-13 5.1776263843335194e-09


KeyboardInterrupt: 

In [ ]:
import torch

In [65]:
params = torch.DoubleTensor(nucl.steps_params)
frames = torch.zeros((params.size()[0],4,4)).double()

In [66]:
%timeit rebuild_base_ref_frames(params,frames)

10.3 ms ± 305 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [98]:
from tqdm.auto import tqdm

In [285]:
for i in tqdm(range(10000)):
    rebuild_base_ref_frames(params,frames)

/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/ipyw

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [94]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
              8751       gpu all_ac_a a_kniaze  R 60-02:58:03      1 node_12
              8887       cpu jup_m_ko m_kovale  R 32-17:37:37      1 node_02
              8929       gpu jup_n_ko n_kosari  R 19-22:28:21      1 node_13
              8932       gpu jup_v_si v_sidoro  R 19-20:10:37      1 node_13
              8933       gpu jup_t_ro t_romano  R 18-22:41:10      1 node_13
              8934       gpu jup_v_va v_vasile  R 17-19:54:05      1 node_13
              8935       gpu jup_n_kr n_kristo  R 17-02:26:23      1 node_13
              8936       gpu jup_a_av a_avakya  R 16-15:52:11      1 node_13
              8938       gpu jup_p_fe p_feskin  R 14-15:29:22      1 node_13
              8939       gpu jup_a_gr a_gribko  R 14-04:06:21      1 node_13
              8941       gpu jup_a_sh a_sharya  R 12-15:18:32      1 node_13
              8944       gpu jup_n_mo n_motori  R 10-16:26:48      1

In [10]:
%timeit pynamod.bp_step_geometry.rebuild_by_full_par_frame_numba(nucl.steps_params)

488 µs ± 33.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
from pynamod.geometry_transformations import *

In [51]:
torch.arange(9).resize(3,3)

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [72]:
bp_frames = rebuild_base_ref_frames(torch.DoubleTensor(nucl.steps_params),frames)

In [ ]:
[[ 9.30957079e-01 -3.57569481e-01 -7.39119925e-02]
  [ 3.59260655e-01  9.33177521e-01  1.05591503e-02]
  [ 6.51973800e-02 -3.63837865e-02  9.97208866e-01]]]

In [45]:
def get_ori_and_mat_from_step_opt(shift,slide,rise,tilt,roll,twist,R1_exp,o1_exp):
    tilt=np.deg2rad(tilt)
    roll=np.deg2rad(roll)
    twist=np.deg2rad(twist)

    #x=np.array([1.,0.,0.])
    y=np.array([0.,1.,0.])
    z=np.array([0.,0.,1.])
    temp1=np.zeros((3,3))
    temp2=np.zeros((3,3))
    
    gamma=np.sqrt(tilt**2 + roll**2)
    #print(gamma)
    #norm=length(np.array([tilt,roll,0]))
    cos_phi =  roll/gamma
    phi = np.arccos(cos_phi)
    if tilt < 0:
        phi *= -1
    #print(phi)
    rmat_inmat(z,twist/2-phi,temp1)
    rmat_inmat(y,gamma/2,temp2)
    temp3=np.dot(temp1,temp2)
    rmat_inmat(z,phi,temp2)   
    
    rm=np.dot(temp3, temp2)
    
    rmat_inmat(y,gamma,temp2)
    temp3=np.dot(temp1,temp2)
    rmat_inmat(z,twist/2+phi,temp2)  
    
    r2=np.dot(temp3, temp2)
    
    o2=np.dot(np.array([shift,slide,rise]),rm.T)
    #print(r2,o2,sep='\n')
    R2_exp = np.dot(R1_exp,r2)
    o2_exp=o1_exp+np.dot(o2,R1_exp.T)
    return(o2_exp,R2_exp)

def rebuild_by_full_par_frame_numba(full_par_frame,start_bp_frame=None):
    
    length=full_par_frame.shape[0]
    bp_frames = np.zeros((length,4,4))
    
    if start_bp_frame is None:
        R1_exp = np.identity(3)
        o1_exp = np.array([0.0,0.0,0.0])
        bp_frames[0,:3,:3] = R1_exp
        bp_frames[0,3,:3] = o1_exp
    else:
        bp_frames[0] = start_bp_frame
        
    for i in range(1,length):
        params=full_par_frame[i]
        bp_frames[i,3,:3],bp_frames[i,:3,:3]=get_ori_and_mat_from_step_opt(params[0],params[1],params[2],
                                                                           params[3],params[4],params[5],
                                                                           bp_frames[i-1,:3,:3],bp_frames[i-1,3,:3])

    return(bp_frames)

In [84]:
from pynamod.geometry.geometry_transformations import *

In [347]:
def quat_mult(a,b):
    res = np.zeros(a.shape)
    res[:,0] = a[:,0] * b[:,0] - a[:,1] * b[:,1] - a[:,2] * b[:,2] - a[:,3] * b[:,3]
    res[:,1] = a[:,0] * b[:,1] + a[:,1] * b[:,0] + a[:,2] * b[:,3] - a[:,3] * b[:,2]
    res[:,2] = a[:,0] * b[:,2] - a[:,1] * b[:,3] + a[:,2] * b[:,0] + a[:,3] * b[:,1]
    res[:,3] = a[:,0] * b[:,3] + a[:,1] * b[:,2] - a[:,2] * b[:,1] + a[:,3] * b[:,0]
    
    return(res)

In [358]:
def quat_mult_old(q1,q2):
    prod = np.zeros(q1.shape)
    prod[:,0] = q2[:,0]*q1[:,0] - np.sum(q1[:,1:]*q2[:,1:],axis=1)
    prod[:,1:] = q2[:,0].reshape(-1,1)*q1[:,1:] + q1[:,0].reshape(-1,1)*q2[:,1:] + np.cross(q1[:,1:],q2[:,1:])
    return prod


def rotate_old(quat,v):
    u = quat[:,1:]
    rot = 2*np.sum(u*v,axis=1).reshape(-1,1)*u 
    rot += (quat[:,0]**2 - np.linalg.norm(u,axis=1)**2).reshape(-1,1)*v
    rot += 2*quat[:,0].reshape(-1,1)*np.cross(u,v)
    return rot

def rotate(quat,v):
    conj = get_conj(quat)
    q_v = np.hstack([np.zeros((quat.shape[0],1)),v])
    prod = quat_mult(quat,q_v)
    return quat_mult(prod,conj)[:,1:]

def get_quat(phi,vec):
    phi = phi.reshape(-1,1)
    vec = np.tile(vec,phi.shape[0]).reshape(-1,3)
    quat = np.hstack([np.cos(phi/2),vec*np.sin(phi/2)])
    return quat

def get_conj(quat):
    return quat*[1,-1,-1,-1]

def normalize(quat):
    sin = np.sin(np.arccos(quat[:,0])).reshape(-1,1)
    vec = quat[:,1:]/sin
    quat[:,1:] /= np.linalg.norm(vec,axis=1).reshape(-1,1)
    return quat   

def quat_from_euler(a,b,g):
    cosa, sina = np.cos(a),np.sin(a)
    cosb, sinb = np.cos(b),np.sin(b)
    cosg, sing = np.cos(g),np.sin(g)
    q = np.zeros((cosa.shape[0],4))
    q[:,3] = cosb*cosa*sing + cosb*sina*cosg
    q[:,2] = sinb*cosa*cosg + sinb*sina*sing
    q[:,1] =  - sinb*cosa*sing + sinb*sina*cosg
    q[:,0] =  cosb*cosa*cosg - cosb*sina*sing
    return q

def rebuild_base_ref_frames_quat(steps_frames,bp_frames):
    steps_frames = np.copy(steps_frames)

    steps_frames[1:,3:] = np.deg2rad(steps_frames[1:,3:])
    z = np.array((0,0,1))
    y = np.array((0,1,0))


    gamma = np.linalg.norm(steps_frames[1:,3:5],axis=1)
    cos_phi =  steps_frames[1:,4]/gamma
    phi = np.arccos(cos_phi)
    phi[steps_frames[1:,3]<0] *= -1
    q1 = get_quat(steps_frames[1:,5]/2 - phi,z)
    qm = quat_mult(q1,get_quat(gamma/2,y))
    qm = quat_mult(qm,get_quat(phi,z))
    
    q2 = quat_mult(q1,get_quat(gamma,y))
    q2 = quat_mult(q2,get_quat(steps_frames[1:,5]/2 + phi,z))
    o2 = rotate(qm,steps_frames[1:,:3])
    bp_frames[0,:4] = [0,0,0,1]
    for i,q in enumerate(q2):
        bp_frames[i+1,:4] = quat_mult(bp_frames[i,:4].reshape(-1,4),q.reshape(-1,4))
    bp_frames[1:,4:] = np.cumsum(rotate(bp_frames[:-1,:4],o2),axis=1)
    
    return bp_frames

In [395]:
%timeit rebuild_base_ref_frames_quat(nucl.steps_params,np.zeros((145,7)))

3.19 ms ± 66.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [364]:
a = np.ones((4,4))
get_conj(a)
a

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [328]:
def rebuild_base_ref_frames(steps_frames,bp_frames):
    steps_frames = np.copy(steps_frames)

    steps_frames[:,3:] = np.deg2rad(steps_frames[:,3:])


    gamma = np.linalg.norm(steps_frames[:,3:5],axis=1)
    cos_phi =  steps_frames[:,4]/gamma
    phi = np.arccos(cos_phi)
    phi[steps_frames[:,3]<0] *= -1
    phi[0] = 0
    rm = Rotation.from_euler('zyz',np.vstack([steps_frames[:,5]/2 - phi,gamma/2,phi]).T)

    r2 = Rotation.from_euler('zyz',np.vstack([steps_frames[:,5]/2 - phi,gamma,steps_frames[:,5]/2 + phi]).T)
    o2 = rm.apply(steps_frames[:,:3])
    print([Rotation.concatenate(r2[:i+1]).apply(o2[i]) for i in range(steps_frames.shape[0])])
    bp_frames[1:,3,:3] = np.cumsum([Rotation.concatenate(r2[:i+1]).apply(o2[i]) for i in range(steps_frames.shape[0])],axis=0).reshape(-1,3)
    
    return bp_frames

In [298]:
def to_quat(steps_params):
    frames = rebuild_by_full_par_frame_numba(steps_params)
    quat = Rotation.from_matrix(frames[:,:3,:3]).as_quat()
    #quat = np.hstack([quat[:,3].reshape(-1,1),quat[:,:3]])
    return np.hstack([quat,frames[:,3,:3]])

In [329]:
rebuild_base_ref_frames(nucl.steps_params,np.zeros((145,4,4)))

[array([[0., 0., 0.]]), array([[ 0.15491968, -0.81949604,  4.05682631],
       [ 0.34882837, -0.24074414,  4.1199241 ]]), array([[0.98870168, 0.21881135, 3.68855256],
       [0.4131591 , 1.05784006, 3.65254752],
       [0.59966788, 0.80051662, 3.69193729]]), array([[-0.497103  ,  0.31548947,  3.03442937],
       [-0.785352  ,  0.18535252,  2.98383532],
       [-0.57973863, -0.1057856 ,  3.0343235 ],
       [-0.10641292,  0.57856162,  3.03452621]]), array([[1.30283043, 0.15637777, 3.1161284 ],
       [0.73858061, 1.1537839 , 3.09117433],
       [0.87633943, 0.96390191, 3.1200961 ],
       [1.52852662, 1.28851047, 2.72679748],
       [0.8366943 , 0.47527646, 3.24131644]]), array([[-0.81750902,  0.14122756,  3.52859044],
       [-0.96836273, -0.10810587,  3.49138991],
       [-0.70474814, -0.45059825,  3.52696867],
       [-0.21878177,  0.36588591,  3.59964981],
       [-0.95161605, -0.72282029,  3.42215933],
       [-1.05123827, -0.2561906 ,  3.45954928]]), array([[-0.19943887,  0.344655

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (145,) + inhomogeneous part.

In [359]:
(rebuild_base_ref_frames_quat(nucl.steps_params,np.zeros((145,7))) - to_quat(nucl.steps_params))[:,4:]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.39998887e-01,  2.20719909e+00,  9.93599823e-01],
       [-1.46666024e+00, -2.16109792e-01, -4.98027625e+00],
       [-6.12157595e-01,  2.42842048e+00, -6.83925442e+00],
       [ 1.22127285e-01, -8.22656704e-01, -1.16818430e+01],
       [-2.18406285e+00,  2.17526696e-01, -1.41883063e+01],
       [-2.64239924e+00,  5.77756701e-01, -1.76025630e+01],
       [-1.37827644e+00,  1.02082189e+00, -2.01212074e+01],
       [-2.64310140e+00,  1.23141324e+00, -2.37356118e+01],
       [-3.75815264e+00, -6.49411010e-02, -2.84917145e+01],
       [-4.93568434e+00,  5.33462644e-01, -3.08101974e+01],
       [-5.92820007e+00, -4.04191749e+00, -3.61759647e+01],
       [-6.80514920e+00, -9.55954205e-01, -3.67637157e+01],
       [-8.99754119e+00, -3.86613721e+00, -4.18438833e+01],
       [-9.58817083e+00, -5.31326628e+00, -4.47916615e+01],
       [-1.35625044e+01, -5.58463017e+00, -4.74044505e+01],
       [-1.45420710e+01, -2.31270144e+00

In [204]:
nucl.steps_params.shape

(145, 6)

### Energy Class

In [ ]:
nucl = pynamod.DNA_structure_from_atomic(pdb_id='3LZ0',leading_strands=['I'],proteins=[])
nucl.analyze_DNA()
nucl.analyze_protein()
nucl.move_to_coord_center()

Sending GET request to https://files.rcsb.org/download/3LZ0.pdb to fetch 3LZ0's pdb file as a string.


In [99]:
nucl2 = pynamod.DNA_structure().append_structures([nucl,dna_gen,nucl_temp])

In [85]:
nucl2.view_structure()

NGLWidget()

In [173]:
class Ouaternion:
    def __init__(self,angle=None,vector=None,quat=None):
        if angle is not None:
            
        
    def quat_mult(q1,q2):
    prod = q1[0]*q2
    prod[1:] += q2[0]*q1[1:]
    prod[0] -= np.sum(q1[1:]*q2[1:])
    prod[1] += q1[2]*q2[3] - q2[2]*q1[3]
    prod[2] += q1[3]*q2[1] - q2[3]*q1[1]
    prod[3] += q1[1]*q2[2] - q2[1]*q1[2]
    return prod

def rotate(quat,v):
    u = quat[1:]
    rot = 2*np.dot(u,v)*u + (quat[0]**2 - np.dot(u,u))*v + 2*quat[0]*np.cross(u,v)
    return rot

def get_quat(phi,vec):
    prod = np.empty(4)
    prod[0] = np.cos(phi/2)
    prod[1:] = vec*np.sin(phi/2)
    return prod

In [41]:
en = pynamod.Energy()

In [156]:
dna_gen = pynamod.DNA_Structure_Generated('atcg'*100)
dna_gen.move_to_torch()

AttributeError: 'DNA_Structure_Generated' object has no attribute 'pairs_list'

In [9]:
from scipy.spatial.distance import cdist

In [23]:
torch.cdist(dna_gen.base_ref_frames[:,3,:3],dna_gen.base_ref_frames[:,3,:3])

tensor([[0.0000e+00, 3.3622e+00, 6.6839e+00,  ..., 7.5125e+03, 7.5158e+03,
         7.5192e+03],
        [3.3622e+00, 1.3811e-03, 3.3506e+00,  ..., 7.5093e+03, 7.5126e+03,
         7.5160e+03],
        [6.6839e+00, 3.3506e+00, 0.0000e+00,  ..., 7.5060e+03, 7.5093e+03,
         7.5127e+03],
        ...,
        [7.5125e+03, 7.5093e+03, 7.5060e+03,  ..., 0.0000e+00, 0.0000e+00,
         6.6332e+00],
        [7.5158e+03, 7.5126e+03, 7.5093e+03,  ..., 0.0000e+00, 0.0000e+00,
         2.0000e+00],
        [7.5192e+03, 7.5160e+03, 7.5127e+03,  ..., 6.6332e+00, 2.0000e+00,
         0.0000e+00]])

In [42]:
en.set_energy_matrices(nucl)

In [11]:
nucl.radii

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2.])

In [68]:
en.add_restraints([Restraint(1,5,1,torch.Tensor((1,1,1,1,1,1)),torch.Tensor((0.5,0.5,0.5,0.5,0.5,0.5)))])

In [44]:
en.get_full_energy(nucl)

tensor(6249.7748, dtype=torch.float64)

In [40]:
en.average_step_params.dtype

torch.float32

In [36]:
m = en._get_real_space_total_energy(dna_gen)

In [10]:
for i in torch.zeros(10,2):
    print(i)

tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])


In [72]:
l = dna_gen.base_ref_frames.shape[0]
dist = torch.zeros(l,l)
for i in range(l):
    for j in range(i,l):
        dist[i,j] = torch.dist(dna_gen.base_ref_frames[i,3,:3],dna_gen.base_ref_frames[j,3,:3])
        dist[j,i] = dist[i,j]

In [78]:
m[1,1]

tensor(0.0014)

In [73]:
dist == m

tensor([[ True, False,  True,  ..., False, False,  True],
        [False, False, False,  ...,  True,  True,  True],
        [ True, False,  True,  ..., False, False,  True],
        ...,
        [False,  True, False,  ...,  True, False, False],
        [False,  True, False,  ..., False,  True, False],
        [ True,  True,  True,  ..., False, False,  True]])

In [35]:
zer = torch.argwhere(m==0)
zer[zer[:,0]!=zer[:,1]]

tensor([[2257, 2258],
        [2258, 2257]])

In [28]:
en.move_to_cuda()

In [29]:
dna_gen.move_to_cuda()

In [30]:
%timeit en.get_full_energy(dna_gen)

78.5 ms ± 2.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
dna_gen.move_to_torch()

In [23]:
from scipy.spatial.distance import pdist

In [25]:
from scipy.spatial.distance import squareform

In [16]:
def set_dist_mat_slice(length):
    sl = torch.arange(length**2).reshape(length,length)
    sl = torch.triu(sl, diagonal=1).reshape(-1)
    return sl[sl>0]

def triform(square_mat,sl):
    return square_mat.reshape(-1)[sl]


In [40]:
import matplotlib.pyplot as plt

In [65]:
from pynamod.geometry.bp_step_geometry import Geometrical_Parameters

In [20]:
class Cl(torch.Tensor):
    def __init__(self,tens_like):
        super().__init__(tens_like)
    
    def h(self):
        print(self.a)
    
    def f(self):
        getattr(self,'h')()

In [21]:
a = Cl(torch.ones(5))

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [15]:
torch.TensorType

[autoreload of pynamod.geometry.bp_step_geometry failed: Traceback (most recent call last):
  File "/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/v_vasilev/.

torch.TensorType

In [118]:
d = Cl(a=[6],b={2:4},d=[])
d.a.append(6)
d.f()

[6, 6]


In [119]:
d.b

{2: 4}

In [391]:
e = Cl()

In [394]:
e.a

[1, 2, 3, 4]

In [396]:
getattr(e,'h')()

[1, 2, 3, 4]


In [26]:
d = {'l':4}

In [27]:
d.l

AttributeError: 'dict' object has no attribute 'l'

In [47]:
torch.stack([torch.from_numpy(np.ones((3,))),torch.from_numpy(np.zeros((3,)))])

tensor([[1., 1., 1.],
        [0., 0., 0.]], dtype=torch.float64)

In [88]:
class A:
    def __init__(self,l=[]):
        self.l = l
        

In [89]:
a = A()

In [48]:
a.l

[]

In [49]:
c = A()

In [50]:
c.l.append(4)

In [92]:
a.l

[5]

In [91]:
a.__dict__['l'] = [5]

In [53]:
a = [1,3]
b = a.copy()

In [54]:
b.append(3)

In [7]:
class mod_Tensor(torch.Tensor):
    def __new__(cls, x,geom_class, *args, **kwargs):
        return super().__new__(cls, x, *args, **kwargs)
    
    def __init__(self,x,geom_class,*args,**kwards):
        self.geom_class = geom_class
    
    def __getitem__(self,sl):
        sl_tens = super().__getitem__(sl)
        sl_tens.geom_class = self.geom_class
        return sl_tens

In [8]:
mod_Tensor(torch.ones(5),'d').geom_class

'd'

In [30]:
a = mod_Tensor(torch.ones(5,5,type=int),'d')

TypeError: ones() received an invalid combination of arguments - got (int, int, type=type), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [31]:
a[3:].geom_class

tensor([1., 1., 1., 1.])


AttributeError: 'mod_Tensor' object has no attribute 'geom_class'

In [2]:
from pynamod.geometry.bp_step_geometry import mod_Tensor

In [9]:
a = mod_Tensor(torch.ones(5,4),'d')

In [10]:
a[2:].geom_class

'd'

In [11]:
'geom_class' == 'geom_class'

True